# Notebook Imports and Initial Setup

In the initial cells, essential libraries and dependencies are imported, setting up the environment for the fine-tuning process. We use a custom trainer, configure GPU settings, and set up necessary libraries like Hugging Face's `transformers` and `peft` (Parameter-Efficient Fine-Tuning).

In [1]:
!pip install --upgrade peft transformers bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 94.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1


In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

In [3]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

**Load Datasets**

In [4]:
from datasets import load_dataset

# Load the Sanskrit dataset from OSCAR
dataset = load_dataset('oscar', 'unshuffled_deduplicated_sa', split='train')

README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [5]:
# Display a few samples

for i in range(3):
    print(f"Sample {i+1}:\n{dataset[i]['text']}\n")

Sample 1:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति । तस्‍य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्‍वयंबर इति चलचित्रं प्रकाश्यते ।
लट् लकार: एकवचनम् द्विवचनम्बहुवचनम्प्रथमपुरुष:गच्‍छतिगच्‍छत:गच्‍छन्तिमध्‍यमपुरुष:गच्‍छसिगच्‍छथ:गच्‍छथउत्‍तमपुरुष:गच्‍छामिगच्‍छाव:गच्‍छाम:

Sample 2:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति; अन्याः संस्थित्यः अपि सन्ति । अधिकं ज्ञातुम् अत्र उपयोगस्य संस्थितिं पश्यतु ।

Sample 3:
स्थिते च कवचे देहे नास्ति मृत्युश्च जीविनाम् । अस्त्रे-शस्त्रे-जले-वह्नौ सिद्धिश्चेन्नास्ति संशयः ॥ ४॥
प्राच्यां मां पातु भूतेशः आग्नेय्यां पातु शङ्करः । दक्षिणे पातु मां रुद्रो नैॠत्यां स्थाणुरेव च ॥ ११॥



**Data Cleaning**

In [6]:
import re

def clean_text(example):
    text = example['text']
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    return {'text': text}

# Apply the cleaning function
dataset = dataset.map(clean_text)

Map:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [7]:
def filter_short_texts(example):
    return len(example['text'].split()) > 5  # Keep texts longer than 5 words

dataset = dataset.filter(filter_short_texts)

Filter:   0%|          | 0/7121 [00:00<?, ? examples/s]

**Load Model**

In [8]:
model_name = '/kaggle/input/gemma-2/transformers/gemma-2-2b/1'

In [9]:
# Use a multilingual tokenizer or train a new one
# Load the tokenizer for Gemma 2 9B model

tokenizer = AutoTokenizer.from_pretrained(model_name)

**Check whether Cuda is available**

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**LORA Configurations**

In [11]:
from transformers import BitsAndBytesConfig

#bnb_config = BitsAndBytesConfig(load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.CAUSAL_LM,
)

In [13]:
model = get_peft_model(model, lora_config)

In [14]:
for name, param in model.named_parameters():
    if 'lora' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

**Number of trainable parameters**

In [15]:
# Verify trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} | All params: {all_params} | "
        f"Trainable percentage: {100 * trainable_params / all_params:.2f}%"
    )

print_trainable_parameters(model)

Trainable params: 6389760 | All params: 1608593664 | Trainable percentage: 0.40%


In [16]:
#model.gradient_checkpointing_enable()

**Prepare Tokenized Dataset**

In [17]:
# Prepare dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
    )
    tokens['labels'] = tokens['input_ids'].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/6957 [00:00<?, ? examples/s]

**Customized Data Collator**

In [18]:
# Custom data collator
def custom_data_collator(features):
    batch = {}
    for key in features[0].keys():
        batch[key] = torch.stack([torch.tensor(f[key]) for f in features]).to('cuda')
    return batch

# Custom Trainer Initialization

A `CustomTrainer` instance is initialized with the model, training arguments, and dataset. This custom trainer coordinates the training process, using the arguments and model components configured earlier.

In [19]:
class CustomTrainer(Trainer):
    def prepare_inputs(self, inputs):
        # Ensure inputs are on GPU
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        return inputs

In [20]:
# Get a batch of data
batch = tokenized_dataset[:2]  # Take the first 2 samples

# Convert batch to tensors and move to GPU
inputs = {
    'input_ids': torch.tensor(batch['input_ids']).to('cuda'),
    'attention_mask': torch.tensor(batch['attention_mask']).to('cuda'),
    'labels': torch.tensor(batch['labels']).to('cuda'),
}

**Set model to evaluation mode**

In [21]:
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    loss = outputs.loss
    print(f"Loss: {loss.item()}")
    print(f"Loss requires grad: {loss.requires_grad}")

Loss: 19.825746536254883
Loss requires grad: False


**Set model to training mode**

In [22]:
model.train()  

outputs = model(**inputs)
loss = outputs.loss

print(f"Loss: {loss.item()}")
print(f"Loss requires grad: {loss.requires_grad}")

Loss: 19.825746536254883
Loss requires grad: True


In [23]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: requires_grad={param.requires_grad}, device={param.device}")

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight: requires_grad=True, device=cuda:0
base_model.model.model.layers.1.self_attn.q_pr

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Setting Training Arguments

Training arguments are configured here, including batch size, number of training epochs, learning rate, and other key settings. This configuration is essential for controlling the fine-tuning process.

In [25]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/results/',
    num_train_epochs=2,
    per_device_train_batch_size=1,  # Adjust based on GPU memory
    gradient_accumulation_steps=32,  # Adjust to maintain effective batch size
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=500,
    gradient_checkpointing=False,
    optim='adamw_bnb_8bit',
    dataloader_pin_memory=False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=custom_data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


# Memory Management and Training Execution

To manage memory effectively on the GPU, we clear the CUDA cache before starting the actual training. Then, the `trainer.train()` command is used to start the fine-tuning.

The training progress, including the number of steps completed, current loss, and other details, is displayed to monitor the model's learning curve.

In [27]:
# Clear cache and start training
torch.cuda.empty_cache()

In [28]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=434, training_loss=1.9588829778855847, metrics={'train_runtime': 14491.896, 'train_samples_per_second': 0.96, 'train_steps_per_second': 0.03, 'total_flos': 8.664651391618253e+16, 'train_loss': 1.9588829778855847, 'epoch': 1.9962627569354607})

# Saving the Trained Model

Once training is complete, we save the fine-tuned model and tokenizer to a directory for future usage in generating Sanskrit text.


In [29]:
trainer.save_model('/kaggle/working/fine-tuned-gemma2-sanskrit-lora')
tokenizer.save_pretrained('/kaggle/working/fine-tuned-gemma2-sanskrit-lora')

('/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer_config.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/special_tokens_map.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer.model',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/added_tokens.json',
 '/kaggle/working/fine-tuned-gemma2-sanskrit-lora/tokenizer.json')

# Model Loading and Evaluation Setup

In this section, we load the fine-tuned model into evaluation mode for generating Sanskrit text. The `PeftModel` allows efficient loading and evaluation.

In [30]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map='auto'
)

model = PeftModel.from_pretrained(model, '/kaggle/working/fine-tuned-gemma2-sanskrit-lora')

# Set the model to evaluation mode
model.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
        

# Text Generation Function

A function, `generate_text`, is defined for generating Sanskrit text based on a provided prompt. Parameters for temperature, top-p sampling, and maximum length control the creativity and diversity of the generated text.


In [31]:
def generate_text(prompt, max_length=512, num_return_sequences=1):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    # Generate output sequences
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,           # Use sampling for more diverse outputs
            temperature=0.7,          # Adjust temperature for creativity
            top_p=0.9,                # Use top-p sampling
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode the generated tokens
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)
    
    return generated_texts

# Example Prompt and Generated Text

Using the `generate_text` function, we provide an example Sanskrit prompt and generate text based on it to showcase the model's capability to create coherent Sanskrit sentences.

In [34]:
prompt = "पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति"
generated_texts = generate_text(prompt, max_length=200)
print("Generated Text:")
print(generated_texts[0])

Generated Text:
पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति । पाठस्य सम्पादकः अनिल मिश्राः। पाठस्य संपादनार्थः अन्येषु इच्छामि । पाठस्य सारः । इच्छामि तस्य स्वरूपस्य विस्तृतपरिचयं करो। विषयः आध्यात्मिकविज्ञानः इतिहासः अर्थशास्त्रः विज्ञानशास्त्रः अर्थशास्त्रः इतिहासः आध्यात्मिकविज्ञानः विज्ञानशास्त्रः अर्थशास्त्रः इतिहासः आध्यात्मिकविज्ञानः विज्ञानशास्त्रः अर्थशास्त्रः इतिहासः आध्यात्मिकविज्ञानः विज्ञानशास्त्रः अर्थशास्त्रः इतिहासः


# Additional Prompt for Text Generation

Further Sanskrit prompts are used to test the model's output diversity and quality, demonstrating the ability to generate text across different contexts in Sanskrit.

In [33]:
prompt = "अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति ।"
generated_texts = generate_text(prompt, max_length=200)

print("Generated Text:")
print(generated_texts[0])

Generated Text:
अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति । प्राग्गुरुगणः अन्‍यः गौरववन्तः अस्ति । अन्‍यः सुप्रसिद्धः । अन्‍यः विद्यमानः अस्ति । अन्‍यः पुष्करसमुद्रम्‍न सह एव गौरववन्तः अस्ति । अन्‍यः क्रीडा सह विद्यमानः अस्ति । अन्‍यः क्रीडापक्षम्‍न सह एव गौरववन्तः अस्ति । अन्‍यः सुप्रसिद्धः । अन्‍यः पुष्करसमुद्रम्‍न सह एव गौरववन्तः अस्ति । अन्‍यः सुप्रसिद्धः । अन्‍यः


# Conclusion

In this notebook, we have successfully fine-tuned the Gemma-2 model using LoRA for generating coherent text in Sanskrit. This approach showcases an efficient way to create and deploy language models in low-resource languages, aiding in language preservation and linguistic research.